## Hierarchical Pointer-Generator Network for Title Generation

There are many variations of the Pointer-Generator network; this implementation was based on the following paper: Get To The Point: <i>Summarization with Pointer-Generator Networks</i>, with the difference being a hierarchical component over the sentence-level embeddings (chosen to be the last vector of a sentence as encoded by the bi-directional RNN). this addition is inspired by the following papers: <i>A Hierarchical Neural Autoencoder for Paragraphs and Documents</i> and <i>Abstractive Text Summarization using Sequence-to-sequence RNNs and Beyond</i>.

In [1]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense,LSTM,Input,RepeatVector,Activation,Softmax,Embedding,Dot
from tensorflow.keras.layers import Softmax,Concatenate,LayerNormalization,Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
import tensorflow as tf
tf.keras.backend.set_floatx('float32')
import numpy as np
import os
import math
import spacy
from collections import Counter

import warnings
warnings.filterwarnings('ignore')
#tf.compat.v1.enable_eager_execution()

In [2]:
tf.__version__ # required due to a bug with tf.gather which previously did not accumulate gradient flow

'2.2.0'

### Data cleaning

In [3]:
token_cutoff=400 # this is the amount to pad up to for the input sequence

In [4]:
# collecting all of the data
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(nlp.create_pipe('sentencizer'))
data_dir = "../../data/bbc_news_summary/news_articles/business/"
files = os.listdir(data_dir)
num_sents = [] # collects the number of sentences present in each occurance, max number of sentences:21
sentence_assignments = [] # assignes a unique number to the sentence assignment for each word

headlines = [] # max length for headlines is 11 + 1 (for the <s>)
body_texts = [] # max length for body text is 400 (imposed)
all_texts = []
for fname in files:
    with open(data_dir+fname) as data_file:
        lines = data_file.readlines()
        lines = [line.strip() for line in lines]
        body = []
        sentence_assignment = []
        headline = lines[0]
        headline = [tok.text.lower() for tok in nlp(headline)]
        doc = nlp(" ".join(lines[1:]))
        for i,sent in enumerate(doc.sents):
            sent_tokens = [tok.text.lower() for tok in nlp(sent.text.strip())]
            sent_len = len(sent_tokens)
            sentence_assignment += [i+1 for _ in range(sent_len)] # index starts at 1, b/c 0 is padding
            body += sent_tokens
        
        body = body[:token_cutoff] # cutting off the length of the body text
        sentence_assignment = sentence_assignment[:token_cutoff]
        headlines.append(headline)
        body_texts.append(body)
        sentence_assignments.append(sentence_assignment)
        num_sents.append(max(sentence_assignment))
        all_texts += body+headline

In [5]:
# getting words which will be part of the fixed_vocabulary (words which appear >= 3 times)
word_freq = Counter(all_texts) # there are 11,727 unique words
words_by_freq = (list(word_freq.items()))
words_by_freq.sort(key=lambda x: x[1],reverse=True) # smaller indices will correspond with more common words
most_freq_words = [word_tup[0] for word_tup in words_by_freq if word_tup[1] >= 3] # 4945 words
less_freq_words = [word_tup[0] for word_tup in words_by_freq if word_tup[1] < 3] # 6782 words
print(len(word_freq),len(most_freq_words),len(less_freq_words))
print(most_freq_words[0:6])
print(less_freq_words[0:6])

11728 4945 6783
['the', '.', ',', 'to', 'of', 'in']
['reinforce', 'sufficiently', 'warming', 'chairmanship', 'jacques', 'thabo']


In [6]:
# assigning indices for all words, and adding <PAD>,<SENT>,<UNK> symbols
# <UNK> both for inputing words not in fixed_vocab and predicting words not in fixed_vocab or in input sequence
word_to_index = {"<PAD>":0,"<SENT>":1,"<UNK>":2} # for all words (including less frequent words)
index_to_word = {0:"<PAD>",1:"<SENT>",2:"<UNK>"}

fixed_vocab_word_to_index = {"<PAD>":0,"<SENT>":1,"<UNK>":2} # for words assigned to the fixed_vocabulary
fixed_vocab_index_to_word = {0:"<PAD>",1:"<SENT>",2:"<UNK>"}

index = 3 # starting index for all words
# assigning indices to most common words:
for word in most_freq_words: 
    word_to_index[word]=index
    index_to_word[index]=word
    fixed_vocab_word_to_index[word]=index
    fixed_vocab_index_to_word[index]=word
    index += 1
    
# assigning indices to least common words:
for word in less_freq_words:
    word_to_index[word]=index
    index_to_word[index]=word
    index += 1

len(fixed_vocab_word_to_index) # there are 4948 words in the fixed_vocabulary

4948

In [7]:
def process_sentence_data(sentence_assignment,num_max_sents=21,token_cutoff=400):
    """ returns all the sentence-level data required for a given example
    """
    num_sents = len(set(sentence_assignment))
    padding_num = num_max_sents-num_sents # amount of padding required
    sent_att_mask = [0 for i in range(num_sents)]
    sent_att_mask += [-np.inf for i in range(padding_num)]
    
    single_sent_indices = [] # indices of end of sentence
    single_sent_lengths = [] # lengths of each sentence
    
    curr_assignment_i = 0
    curr_assignment = sentence_assignment[0]
    for i,assignment in enumerate(sentence_assignment): # denotes the integer representation of the sentence assignment of curr word
        if assignment != curr_assignment:
            single_sent_lengths.append(i-curr_assignment_i)
            single_sent_indices.append(i-1)
            curr_assignment = assignment
            curr_assignment_i = i
            
    single_sent_lengths.append(i-curr_assignment_i+1) # +1 necessary for index ordering
    single_sent_indices.append(i-1+1)
    
    single_sent_indices += [0 for _ in range(padding_num)] # padding
    single_sent_lengths += [0 for _ in range(padding_num)]
    
    num_words = sum(single_sent_lengths)
    if num_words < token_cutoff: # ensures that sentence-level attention will be the same size as word-level attention
        amount_to_pad = token_cutoff-num_words
        single_sent_lengths[-1]=amount_to_pad
    assert(sum(single_sent_lengths)==token_cutoff)
    
    return sent_att_mask,single_sent_indices,single_sent_lengths

In [8]:
# creating the input data representations for the model - input is padded up to a length of token_cutoff
x = [] # stores the integer/index representation for all input
x_indices = [] # stores the joint probability vector indices for all words in the input 
x_indices_dicts = [] # stores the dicts for assigning words which are not in the fixed_vocabulary
att_mask = [] # stores the attention masks (0 for valid words, -np.inf for padding)

# new things for the sentence-level hierarchical model:
sent_att_masks = [] # mask used for padding the sentence embeddings (makes word-level att. mask obsolete)
sent_indices = [] # index of the last vector for each sentence (padded up to length 21)
sent_lengths = [] # the length of each of the sentences

for bt_i,body_text in enumerate(body_texts): # processing the input
    # processing the word-level data:
    x_rep = []
    for token in body_text:
        if token in fixed_vocab_word_to_index:
            x_rep.append(fixed_vocab_word_to_index[token])
        else:
            x_rep.append(fixed_vocab_word_to_index['<UNK>'])
    
    att_mask_rep = [0 for i in range(len(x_rep))]
    amount_to_pad = token_cutoff-len(x_rep)
    x_rep += [0 for i in range(amount_to_pad)] # padding the input
    att_mask_rep += [-np.inf for i in range(amount_to_pad)]
    x.append(x_rep)
    att_mask.append(att_mask_rep)
    
    index = 4948 # starting index for assignment to joint_probability vector
    non_vocab_dict = {}
    this_x_indices = []
    for token in body_text: # assigning each word an index in the joint_probability vector
        if token in fixed_vocab_word_to_index:
            this_x_indices.append(fixed_vocab_word_to_index[token])
        else:
            if token in non_vocab_dict: # this word if OOV but has been seen before
                this_x_indices.append(non_vocab_dict[token])
            else: # this word is OOV and has never been seen before
                non_vocab_dict[token]=index
                this_x_indices.append(index)
                index += 1
                
    x_indices_dicts.append(non_vocab_dict)
    this_x_indices += [0 for i in range(amount_to_pad)] # padding will be masked out in att calculation, so padding with 0 here is valid
    x_indices.append(this_x_indices)
    
    # processing the sentence-level data:
    sentence_assignment = sentence_assignments[bt_i]
    sent_att_mask,single_sent_indices,single_sent_lengths = process_sentence_data(sentence_assignment)
    sent_att_masks.append(sent_att_mask)
    sent_indices.append(single_sent_indices)
    sent_lengths.append(single_sent_lengths)

In [9]:
# this is the largest number of OOV words for a given bid utterances
max([len(dic) for dic in x_indices_dicts])

57

In [10]:
# creating the output representations for the model - output is padded up to a length of 11+1 (for final <s> prediction)
## all words in output that are not in input utterances or in fixed_vocab_vector are assigned 3:<UNK>
y = [] # stores the index representations for all words in the headlines
loss_mask = [] # 1 for valid words, 0 for padding
decoder_x = [] # starts with 1:<SENT>, followed by y[0:len(headline)-1] (this is the input for teacher-forcing)(12x1)
y_indices = [] # index for the correct decoder prediction, in the joint-probability vector

for hl_i,headline in enumerate(headlines): # processing the output
    
    y_rep = [] # not used in the model, stores indices using only fixed_vocab_vector
    for token in headline:
        if token in fixed_vocab_word_to_index:
            y_rep.append(fixed_vocab_word_to_index[token])
        else:
            y_rep.append(fixed_vocab_word_to_index['<UNK>'])
    y_rep.append(fixed_vocab_word_to_index['<SENT>']) # end delimiter of output representation
    
    loss_mask_rep = [1 for i in range(len(y_rep))]
    decoder_x_rep = [1]+y_rep[0:len(y_rep)-1] # embedding word in input but not in fixed_vocab is currently set to <UNK>
    amount_to_pad = 12-len(y_rep) # 11+1 represents final <SENT> prediction
    y_rep += [0 for i in range(amount_to_pad)]
    loss_mask_rep += [0 for i in range(amount_to_pad)] # cancels out loss contribution from padding
    decoder_x_rep += [0 for i in range(amount_to_pad)]
    
    # creating joint-probability representation of output:
    non_vocab_dict = x_indices_dicts[hl_i]
    y_indices_rep = []
    for token in headline:
        if token in fixed_vocab_word_to_index: # word is in fixed_vocabulary
            y_indices_rep.append(fixed_vocab_word_to_index[token])
        elif token in non_vocab_dict: # word is OOV but in the input utterances, use the index assigned to this word in x_indices
            y_indices_rep.append(non_vocab_dict[token])
        else: # word is OOV and not in input utterances
            y_indices_rep.append(fixed_vocab_word_to_index["<UNK>"])
    
    y_indices_rep.append(fixed_vocab_word_to_index['<SENT>']) # last prediction should be <SENT>
    y_indices_rep += [0 for i in range(amount_to_pad)] # padding ignored due to loss_mask
    y.append(y_rep)
    loss_mask.append(loss_mask_rep)
    decoder_x.append(decoder_x_rep)
    y_indices.append(y_indices_rep)

In [11]:
x = np.array(x)
x_indices = np.array(x_indices)
att_mask = np.array(att_mask)
print(x.shape,x_indices.shape,att_mask.shape)
loss_mask = np.array(loss_mask)
decoder_x = np.array(decoder_x)
y_indices = np.array(y_indices)
print(loss_mask.shape,decoder_x.shape,y_indices.shape)
sent_att_masks = np.array(sent_att_masks)
sent_indices = np.array(sent_indices)
sent_lengths = np.array(sent_lengths)
print(sent_att_masks.shape,sent_indices.shape,sent_lengths.shape)

(510, 400) (510, 400) (510, 400)
(510, 12) (510, 12) (510, 12)
(510, 21) (510, 21) (510, 21)


In [12]:
x = x.astype("int32")
x_indices = x_indices.astype("int32")
att_mask = att_mask.astype("float32")
loss_mask = loss_mask.astype("int32")
decoder_x = decoder_x.astype("int32")
y_indices = y_indices.astype("int32")
sent_att_masks = sent_att_masks.astype("float32")
sent_indices = sent_indices.astype("int32")
sent_lengths = sent_lengths.astype("int32")

### Pointer-gen architecture code

In [13]:
def apply_scatter_nd(updates,indices,batch_size):
    """ applies scatter_nd over the batch dimension
    """
    return tf.convert_to_tensor([tf.scatter_nd(indices[i],updates[i],tf.constant([5100],dtype=tf.int32)) for i in range(batch_size)]) # assuming a max vocab_size+unique_words_in_input of 4948+102

In [14]:
def pointer_gen_network(embedding_layer,att_w1,att_w2,att_w3,att_v,sent_att_w1,sent_att_w2,sent_att_v,vocab_d,pgen_w1,pgen_w2,pgen_w3,decoder_lstm,encoder_h=128,input_len=400,output_len=12,max_num_sents=21,batch_size=30):
    """ Returns pointer generator network using Transformer encoder
    args:
        input_len: the length of the input sequence (to the encoder)
        output_len: the length of the output sequence (from the decoder)
        batch_size: cannot be inferred so must be explicitly inputted
        max_num_sents: the maximum number of unique sentences
    """
    x = Input(shape=(input_len),dtype=tf.int32) # input to the encoder
    x_indices_ = Input(shape=(input_len),dtype=tf.int32) # represents where each input word prob. should be added in joint prob. vector
    x_indices = tf.expand_dims(x_indices_,axis=-1)
    att_mask = Input(shape=(input_len)) # mask used with the attention distribution to mask out padding
    decoder_x = Input(shape=(output_len),dtype=tf.int32) # delayed y_data for input to the decoder (for teacher-forcing)
    y_indices = Input(shape=(output_len),dtype=tf.int32) # indices of the correct word in the joint_probabilities vector
    sent_indices = Input(shape=(max_num_sents),dtype=tf.int32) # sentence-lvl attention components
    sent_lengths = Input(shape=(max_num_sents),dtype=tf.int32)
    sent_att_mask = Input(shape=(max_num_sents))
    s = tf.zeros((batch_size,256),dtype=tf.float32) # defining using batch_size makes model brittle, but fine for training
    c = tf.zeros((batch_size,256),dtype=tf.float32)
    coverage_vector = tf.zeros((batch_size,input_len),dtype=tf.float32)
    
    input_e = embedding_layer(x) # embeddings for the input
    h = Bidirectional(LSTM(encoder_h,activation="tanh",return_sequences=True),merge_mode="concat")(input_e) # word-level encoder
    sentence_h = tf.gather(h,sent_indices,batch_dims=1) # getting sentence-level representations
    
    decoder_e = embedding_layer(decoder_x) # embeddings for delayed input to the decoder
    outputs = []
    coverage_loss_contributions = [] # stores coverage loss contribution for each decoder output step
    
    fixed_vocab_indices = tf.tile(tf.reshape(tf.range(4948,dtype=tf.int32),(1,4948,1)),tf.constant([batch_size,1,1])) # 4948 is fixed_vocab size
    
    for i in range(output_len): # loop through each step of the decoder
        decoder_input = decoder_e[:,i,:]  # input to the decoder at this timestep
        s,_,c = decoder_lstm(tf.expand_dims(decoder_input,axis=1),initial_state=[s,c])
        
        # calculating sentence-level attention:
        s_rep = RepeatVector(max_num_sents)(s) # copying the decoder hidden state
        sent_e = sent_att_v(Activation("tanh")(sent_att_w1(sentence_h)+sent_att_w2(s_rep)))
        sent_e = tf.squeeze(sent_e,axis=-1)+sent_att_mask
        sent_a = Activation("softmax")(sent_e) # scaled sentence-level attention
        sent_a = tf.convert_to_tensor([tf.repeat(sent_a[i],sent_lengths[i],axis=0) for i in range(batch_size)]) # get to same shape as word-lvl attention
        
        # calculating word-level attention (probabilities over input tokens):
        s_rep = RepeatVector(input_len)(s) # copying the decoder hidden state
        e = att_v(Activation("tanh")(att_w1(h)+att_w2(s_rep)+att_w3(tf.expand_dims(coverage_vector,axis=-1)))) # unscaled attention
        e = tf.squeeze(e,axis=-1)+att_mask # using attention mask (masks out padding in the input sequence)
        a = Activation("softmax")(e) # scaled attention (represents prob. over input)
        
        # calculating hierarchical attention:
        a = a*sent_a # updating word-level attention based on sentence-level attention
        a = Activation("softmax")(a)
        
        # handling coverage vector computations:
        step_coverage_loss = tf.reduce_sum(tf.minimum(coverage_vector,a),axis=-1) # cov loss at this decoder step
        coverage_loss_contributions.append(step_coverage_loss)
        coverage_vector+=a
        
        # calculating probabilities over fixed vocabulary:
        context = Dot(axes=1)([a,h]) # calculating the context vector
        pre_vocab_prob = Concatenate()([s,context]) # 512 dimentional
        pre_vocab_prob = vocab_d(pre_vocab_prob) # could reduce memory here by first reducing this to < 512 dim. with a linear layer
        vocab_prob = Activation("softmax")(pre_vocab_prob)
        
        # calculation probabilty for text generation:
        pre_gen_prob = pgen_w1(context)+pgen_w2(s)+pgen_w3(decoder_input)
        gen_prob = Activation("sigmoid")(pre_gen_prob)
    
        # calculating joint-probability for generation/copying:
        vocab_prob *= gen_prob # probability of generating a word from the fixed vocabulary
        copy_prob = a*(1-gen_prob) # probability of copying a word from the input
        
        vocab_prob_projected = apply_scatter_nd(vocab_prob,fixed_vocab_indices,batch_size)
        copy_prob_projected = apply_scatter_nd(copy_prob,x_indices,batch_size)
        joint_prob = vocab_prob_projected+copy_prob_projected
        
        # gathering predictions from joint-probability vector - doing it here will reduce memory consumption
        y_indices_i = tf.expand_dims(y_indices[:,i],axis=-1) # getting predictions at time i for whole batch
        predictions_i = tf.squeeze(tf.gather(joint_prob,y_indices_i,batch_dims=1,axis=-1),axis=-1)
        outputs.append(predictions_i)
    
    outputs = K.permute_dimensions(tf.convert_to_tensor(outputs),(1,0))
    coverage_loss_contributions = K.permute_dimensions(tf.convert_to_tensor(coverage_loss_contributions),(1,0))
    
    model = Model(inputs=[x,x_indices_,decoder_x,att_mask,y_indices,sent_indices,sent_lengths,sent_att_mask],outputs=[outputs,coverage_loss_contributions])
    return model

In [15]:
def loss_function(prediction_probabilities,loss_mask,coverage_loss,lam=0.1,use_coverage_loss=True):
    """ Returns the loss for this batch
    args:
        prediction_probabilities: model-assigned probabilities for ground-truth predictions
        loss_mask: vector of 1s,0s specifying whether an input should contribute to the loss
        coverage_loss: coverage loss for this batch of examples
        lam: hyperparameter determining the contribution of coverage_loss to overall loss
        use_coverage_loss: whether coverage loss should be used
    """
    p_words = -tf.math.log(prediction_probabilities)
    p_words *= loss_mask # applying the loss mask
    p_words = tf.reduce_sum(p_words,axis=-1)
    general_loss_component = tf.reduce_mean(p_words)
    
    # incorporating the coverage loss:
    coverage_loss_component = 0
    if use_coverage_loss:
        coverage_loss *= loss_mask # applying the loss mask
        coverage_loss = tf.reduce_sum(coverage_loss,axis=-1)
        coverage_loss_component = lam*tf.reduce_mean(coverage_loss)
        
    total_loss = general_loss_component+coverage_loss_component
    return total_loss

In [16]:
embedding_layer = Embedding(input_dim=4950,output_dim=128,mask_zero=True) # re-used for both the encoder and decoder
decoder_h=256
decoder_lstm = LSTM(decoder_h,activation="tanh",return_state=True)
att_w1 = Dense(256,use_bias=True,activation=None) # for word-level attention
att_w2 = Dense(256,use_bias=True,activation=None)
att_w3 = Dense(256,use_bias=True,activation=None)
att_v = Dense(1,use_bias=False,activation=None)
sent_att_w1 = Dense(256,use_bias=True,activation=None) # for sentence-level attention
sent_att_w2 = Dense(256,use_bias=True,activation=None)
sent_att_v = Dense(1,use_bias=False,activation=None)
vocab_d = Dense(4948,use_bias=True,activation=None) # 4948 is fixed_vocabulary size
pgen_w1 = Dense(1,use_bias=True,activation=None)
pgen_w2 = Dense(1,use_bias=True,activation=None)
pgen_w3 = Dense(1,use_bias=True,activation=None)

In [17]:
batch_size=10
model = pointer_gen_network(embedding_layer,att_w1,att_w2,att_w3,att_v,sent_att_w1,sent_att_w2,sent_att_v,vocab_d,pgen_w1,pgen_w2,pgen_w3,decoder_lstm,encoder_h=128,batch_size=batch_size)
optimizer = Adam(lr=0.01)

In [18]:
# training the model

for _ in range(10): # epochs
    losses = []
    for i in range(0,100-batch_size,batch_size): # only using first 100 samples for training
        x_subset = x[i:i+batch_size]
        x_indices_subset = x_indices[i:i+batch_size]
        decoder_x_subset = decoder_x[i:i+batch_size]
        att_mask_subset = att_mask[i:i+batch_size]
        y_indices_subset = y_indices[i:i+batch_size]
        loss_mask_subset = loss_mask[i:i+batch_size]
        sent_indices_subset = sent_indices[i:i+batch_size]
        sent_lengths_subset = sent_lengths[i:i+batch_size]
        sent_att_mask_subset = sent_att_masks[i:i+batch_size]
        
        with tf.GradientTape() as tape:
            prediction_probabilities,coverage_loss = model([x_subset,x_indices_subset,decoder_x_subset,att_mask_subset,y_indices_subset,sent_indices_subset,sent_lengths_subset,sent_att_mask_subset])
            loss = loss_function(prediction_probabilities,loss_mask_subset,coverage_loss,lam=0.1,use_coverage_loss=True)
        losses.append(float(loss))
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(sum(losses)/max(len(losses),1))

45.100582122802734
34.27932548522949
31.368357552422417
30.25632158915202
29.239930894639755
28.66851086086697
26.980822457207573
24.771902508205837
22.86968782212999
20.986548105875652
